In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
import time as tm
import numpy as np

ModuleNotFoundError: No module named 'numpy'

In [ ]:
Z_trn = np.loadtxt("secret_trn.txt")
Z_tst = np.loadtxt("secret_tst.txt")

n_trials = 5

d_size = 0
t_train = 0
t_map = 0

acc0 = 0
acc1 = 0

In [ ]:
def my_map(challenges):
    features = []

    for challenge in challenges:
        # Take the first 32 numbers of the challenge array
        challenge_subset = challenge[:32]

        di = 1 - 2 * challenge_subset  # Calculate di values for the subset of challenge

        # Calculate cumulative product in reverse
        cumprod_di = np.cumprod(di[::-1])[::-1]

        # Calculate xi values using cumulative products
        xi = cumprod_di[:-1]  # Remove the last element

        feature_vector = np.concatenate((xi, di)).flatten()  # Concatenate and flatten xi and di
        features.append(feature_vector)

    feat = np.array(features)  # Convert list of feature vectors to numpy array
    return feat



from sklearn.svm import LinearSVC

def my_fit(X_train, y0_train, y1_train):
################################
#  Non Editable Region Ending  #
################################
    # Use this method to train your models using training CRPs
    # X_train has 32 columns containing the challenge bits
    # y0_train contains the values for Response0
    # y1_train contains the values for Response1

    # Set C values based on the provided parameter

    C=0.01
    # Map challenges to features
    features_train = my_map(X_train)

    # Train models for Response0 using LinearSVC with specified loss and C value


    # Train models for Response0 using LinearSVC with specified loss and C value
    model0 = LinearSVC( C=C)
    model0.fit(features_train, y0_train)
    w0 = model0.coef_.flatten()
    b0 = model0.intercept_[0]

    # Train models for Response1 using LinearSVC with specified loss and C value
    model1 = LinearSVC(C=C)
    model1.fit(features_train, y1_train)
    w1 = model1.coef_.flatten()
    b1 = model1.intercept_[0]


    return w0, b0, w1, b1


In [ ]:
for t in range(n_trials):

    tic = tm.perf_counter()
    w0, b0, w1, b1 = my_fit(Z_trn[:, :-2], Z_trn[:, -2], Z_trn[:, -1])
    toc = tm.perf_counter()

    t_train += toc - tic
    w0 = w0.reshape(-1)
    w1 = w1.reshape(-1)

    d_size += max(w0.shape[0], w1.shape[0])

    tic = tm.perf_counter()
    feat = my_map(Z_tst[:, :-2])
    toc = tm.perf_counter()
    t_map += toc - tic

    scores0 = feat.dot(w0) + b0
    scores1 = feat.dot(w1) + b1

    pred0 = np.zeros_like(scores0)
    pred0[scores0 > 0] = 1
    pred1 = np.zeros_like(scores1)
    pred1[scores1 > 0] = 1

    acc0 += np.average(Z_tst[:, -2] == pred0)
    acc1 += np.average(Z_tst[:, -1] == pred1)

In [ ]:
d_size /= n_trials
t_train /= n_trials
t_map /= n_trials
acc0 /= n_trials
acc1 /= n_trials

In [ ]:
print( f"{d_size},{t_train},{t_map},{acc0},{acc1}" )